In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [ ]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE

In [ ]:
tf.keras.backend.set_floatx('float64')

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(2,)),
        layers.Dense(2, name="layer1"),
    ]
)


In [ ]:
import neural_ode.ODESolvers

In [ ]:
n_ode = neural_ode.NeuralODE.NeuralODE(model, 2, 
                                       solver=neural_ode.ODESolvers.BackwardEulerMethod())

In [ ]:
n_ode.pretrain()

In [ ]:
# Generate solution

In [ ]:
#
N_n = int(2)
c = 0.1
k = 4.0
def oscilator(t, y):
    return np.array([y[1], -c*y[1]-k*y[0]])
t_final = 20.0
n_eval = int(500)
t_span = np.array([0.0, t_final])
y0 = np.array([1.0, 0.0])

In [ ]:
sol = solve_ivp(oscilator, t_span, y0, t_eval=np.linspace(0, t_final, num=n_eval))

In [ ]:
plt.plot(sol.t, sol.y[0,:])

In [ ]:
# transform to tensorflow
t_span_tf = tf.constant(t_span)
y0_tf = tf.constant(y0, dtype=tf.float64)
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )

In [ ]:
sol1 = n_ode.forward_solve(t_target[0:3], y_target[0,:])
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

In [ ]:
# adjoint method

In [ ]:
loss, dl_dy, a = n_ode.adjoint_method(t_target[0:3], y_target[0:3,:])

In [ ]:
# fit

In [ ]:
n_epoch = 40
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_fold=5, adjoint_method=False)

In [ ]:
# Check derivatives

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss, dl_dp = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)
#loss, dL_dy, a = n_ode.adjoint_method(t_target, y_target, x_external=f_ext_interp)
#dl_dp = a[2:]

In [ ]:
dp = 0.00001
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c+dp], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss2, dl_dp2 = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)

In [ ]:
(loss2-loss)/dp

In [ ]:
dl_dp